In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
from __future__ import print_function
import os
from io import BytesIO
from functools import partial
import PIL.Image
import scipy.misc

In [3]:
graph = tf.Graph()
model_fn = 'tensorflow_inception_graph.pb'
sess = tf.InteractiveSession(graph=graph)

In [4]:
with tf.gfile.FastGFile(model_fn,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [5]:
t_input=tf.placeholder(np.float32,name='input')
imagenet_mean=117.0
t_preprocessed=tf.expand_dims(t_input - imagenet_mean,0)
tf.import_graph_def(graph_def,{'input':t_preprocessed})

In [6]:
layers = [op.name for op in graph.get_operations()if op.type == 'Conv2D']
#输出卷积层层数

print('Number of layers',len(layers))

Number of layers 59


In [ ]:
#定义卷积层，并取出对应的tensor
name = 'mixed4c'
#name = 'mixed4d_3x3_bottleneck_pre_relu'
channel = 118
layer_output = graph.get_tensor_by_name("import/%s:0" % name)
#layer_output[:,:,:,channel]即可表示该卷积层的第140个通道
print(layer_output)

In [ ]:
def savearray(img_array,img_name):
    scipy.misc.imsave(img_name,img_array)
    print('img saved:%s' % img_name)
    
def resize_ratio(img,ratio):
    min = img.min()
    max = img.max()
    img = (img-min)/(max-min)*255
    img = np.float32(scipy.misc.imresize(img,ratio))
    img = img/255*(max-min)+min
    return img
    
def resize(img,hw):
    min = img.min()
    max = img.max()
    img = (img-min)/(max-min)*255
    img = np.float32(scipy.misc.imresize(img,hw))
    img = img/255*(max-min)+min
    return img

def render_naive(t_obj, img0, iter_n=20, step=1.0):
    '''
    渲染函数
    :param t_obj:卷积层某个通道的值
    :param img0:初始化图像
    :param iter_n:迭代的步数
    :param step:
    :return:
    '''
    # t_score 是优化目标，他是t_obj的平均值
    # t_score 越大，就说明神经网络卷积层对应的通道的平均激活越大
    t_score = tf.reduce_mean(t_obj)
    # 计算t_score对t_input的梯度
    # 代码的目标是通过调整输入图像 t_input ，来让 t_score 尽可能的大
    # 因此使用体服下降法
    t_grad = tf.gradients(t_score, t_input)[0]

    # 创建新图
    img = img0.copy()
    # 迭代 iter_n 每一步都将梯度应用到图像上
    for i in range(iter_n):
        # 在sess中计算梯度，以及当前的score
        g, score = sess.run([t_grad, t_score], {t_input: img})
        # 对img应用梯度，step可以看作学习率
        g /= g.std() + 1e-8
        img += g * step
        print('iter:%d' %(i + 1),'score(mean)=%f' % (score))

    savearray(img, 'mountain_deepdream.jpg')

In [ ]:
#img_test = PIL.Image.open("mountain.jpg") 
#定义图像噪声
#img_noise = np.random.uniform(size=(224,224,3)) + 100.0

#调用render_naive函数渲染

#render_naive(layer_output,img_test,iter_n = 100)
render_naive(layer_output,img_noise,iter_n = 20)

In [ ]:
#保存并显示图片

im = PIL.Image.open('navie_deepdream.jpg')
im.show()
#im.save("mountain_naive.jpg")
#im = PIL.Image.open('naive_deepdream.jpg')
#im.show()
im.save('shallow_single_chn.jpg')

In [7]:
def calc_grad_tiled(img,t_grad,tile_size=512):
    sz=tile_size
    h,w=img.shape[:2]
    sx,sy=np.random.randint(sz,size=2)
    img_shift=np.roll(np.roll(img,sx,1),sy,0)
    grad=np.zeros_like(img)
    for y in range(0,max(h-sz//2,sz),sz):
        for x in range(0,max(w-sz//2,sz),sz):
            sub = img_shift[y:y+sz,x:x+sz]
            g=sess.run(t_grad,{t_input:sub})
            grad[y:y+sz,x:x+sz]=g
    return np.roll(np.roll(grad,-sz,1),-sy,0)

In [12]:
def savearray(img_array,img_name):
    scipy.misc.imsave(img_name,img_array)
    print('img saved:%s' % img_name)
    
def resize_ratio(img,ratio):
    min = img.min()
    max = img.max()
    img = (img-min)/(max-min)*255
    img = np.float32(scipy.misc.imresize(img,ratio))
    img = img/255*(max-min)+min
    return img
    
def resize(img,hw):
    min = img.min()
    max = img.max()
    img = (img-min)/(max-min)*255
    img = np.float32(scipy.misc.imresize(img,hw))
    img = img/255*(max-min)+min
    return img

def render_deepdream(t_obj,img0,iter_n=10,step=1.5,octave_n=4,octave_scale=1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score,t_input)[0]
    img=img0.copy()
    
    octaves=[]
    for i in range(octave_n-1):
        hw=img.shape[:2]
        lo = resize(img,np.int32(np.float32(hw)/octave_scale))
        hi = img - resize(lo,hw)
        img = lo
        octaves.append(hi)
    for octave in range(octave_n):
        if octave>0:
            hi = octaves[-octave]
            img=resize(img,hi.shape[:2])+hi
        for i in range(iter_n):
            g = calc_grad_tiled(img,t_grad)
            img += g*(step/(np.abs(g).mean()+1e-7))
    img = img.clip(0,255)
    savearray(img,'mountain_deepdream.jpg')
    im = PIL.Image.open('mountain_deepdream.jpg').show()

In [ ]:
name = 'mixed4c'
layer_output = graph.get_tensor_by_name("import/%s:0" % name)

img0=PIL.Image.open('mountain.jpg')
img0=np.float32(img0)
render_deepdream(tf.square(layer_output),img0)

D:\Anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
